In [1]:
import pandas as pd

In [2]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()

    com = []
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0

    return com

def seperate_comma(com_lst):
    raw_lst = [[ele if ele[-1] != ',' else ele[:-1] + ' ' + ',' for ele in command.split()] for command in com_lst]
    com_lst = [' '.join(ele) for ele in raw_lst]
    return com_lst

def read_all(num_files):
    com_lst = []
    for i in range(1,num_files+1):
        path = './Examples/EGPSR Examples_' + str(i)+ '.txt'
        # path = './Examples/EGPSR Examples.txt'
        com_lst.extend(read_command(path))

    com_lst = seperate_comma(com_lst)
    return com_lst

num_files = 10
# num_files = 1

com_lst = read_all(num_files)
# print(com_lst[3])

print('len: ',len(com_lst))
print(com_lst[:10])

len:  99993
['Could you please tell -B_tell- me -B_tell_dest- how the person -B_tell_what- standing -I_tell_what- at -I_tell_what- the -I_tell_what- entrance -I_tell_what- looks like', 'Please offer -B_take- something -B_take_obj- to -I_take_obj- drink -I_take_obj- to all -B_take_dest- the -I_take_dest- children -i- pointing -i- to -i- the -i- left -I_take_dest- in the dining -i- room -B_take_dest-', 'Handshake -B_greet- the youngest -i- man -B_greet_per- in the dining -i- room -B_greet_dest- please', 'Locate -B_find- a dish -B_find_obj- in the bedroom -B_find_dest- , locate -B_find- Patricia -B_find_per- in the kitchen -B_find_dest- , and say -B_tell- something -B_tell_what-', 'Offer -B_take- something -B_take_obj- to -I_take_obj- drink -I_take_obj- to all -B_take_dest- the -I_take_dest- elders -i- wearing -i- sweaters -i- -I_take_dest- in the kitchen -B_take_dest- please', 'Could you tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- sitting -i- men -I_tell_what- the

In [3]:
for i,s in enumerate(com_lst):
   if i == 1000:
       break
   if 'me -B_tell_dest- how -B_tell_what-' in s:
       print(s)

Could you tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- sitting -i- men -I_tell_what- there -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- kitchen -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- cleaning -i- stuff -I_tell_what- items -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- storage -i- table -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- women -I_tell_what- in -I_tell_what- the -I_tell_what- kitchen -I_tell_what- are -I_tell_what- wearing -i- jackets -i- -I_tell_what- please
Could you please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- yellow -I_tell_what- tableware -I_tell_what- objects -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- counter -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- men -i- are -i- raising -i- their -i- left -i- arm -I_tell_what- in -I_tell_what- the -I

In [4]:
def find_sub_list(sub_list,this_list):
    start = this_list.index(sub_list[0])
    return (start,start+len(sub_list))

def process_person_pro(com_lst,path):
    lines = [line.strip() for line in open(path) if line.strip() != '' and line.strip() != 'it']
    pairs_lst = [(lines[i].split(' '),lines[i+1]) for i in range(0,len(lines)//2,2)] #[([B-referee, I-referee],referral),([B-referee, I-referee],referral),....]
    # print(pairs_lst)
    person_pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        lst = com.split()
        cur_referee  = pairs_lst[0][0]
        cur_referral = pairs_lst[0][1]

        words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
        str_referee = ' '.join(cur_referee).strip()
        if str_referee in words_lst and cur_referral in words_lst:
            # ------------ store command idx ------------
            person_pro_dict['idx'].append(i)

            # ------------ store original command  ------------
            person_pro_dict['words'].append(' '.join(words_lst))

            # ------------ store referral, the pronoun in word command ------------
            referral_idx = words_lst.index(cur_referral)
            person_pro_dict['referral'].append((referral_idx,cur_referral))

            # ------------ store referee, the word being referred ------------
            referee_idx = find_sub_list(cur_referee,words_lst)
            #print(referee_idx)
            person_pro_dict['referee'].append((referee_idx,str_referee))

            pairs_lst.pop(0)
            if len(pairs_lst) == 0:
                break
    return person_pro_dict

# path = './Examples/pros_test.txt'
path = './Examples/pros.txt'

person_pro_dict = process_person_pro(com_lst,path)
per_df = pd.DataFrame(person_pro_dict)
per_df

,idx,referee,referral,words
0,12,"((7, 8), Patricia)","(11, her)","Spot Charlie in the bedroom , find Patricia , ..."
1,57,"((1, 2), William)","(8, him)","Contact William at the entrance , then greet him"
2,74,"((5, 6), Skyler)","(12, her)","Locate a sausages , pinpoint Skyler in the cor..."
3,81,"((8, 9), Charlie)","(12, her)","Robot please navigate to the couch , pinpoint ..."
4,83,"((2, 3), Alex)","(9, him)","Look for Alex in the dining room , accompany h..."
...,...,...,...,...
6677,50257,"((1, 2), Charlie)","(7, her)","Pinpoint Charlie at the sink , conduct her to ..."
6678,50258,"((7, 8), Skyler)","(15, him)","Please take the paprika , look for Skyler at t..."
6679,50259,"((6, 7), Charlie)","(10, him)","Navigate to the bookcase , locate Charlie , an..."
6680,50261,"((5, 6), James)","(12, him)","Please find Elizabeth , find James in the corr..."


In [5]:
#{'_178-it_', '_285-it_', '_226-it_', '_226-it_,', '_95-it_', '_286-it_', '_227-it_', '_179-its_'}
def get_it_s_lab(com_lst):
    pro_tag_set = set()
    for com in com_lst:
        lst = com.split()
        for ele in lst:
            if ele[0] == '_':
                pro_tag_set.add(ele)
    return pro_tag_set



#"referral" : The referring wildcard.
#"referee" : The wildcard being referred.
def store_it_s_coref(com_lst):
    clean_com_lst = com_lst.copy()
    pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    pro_tag_set = get_it_s_lab(com_lst)
    for i,com in enumerate(com_lst):

        # if i == 50:
        #     break

        lst = com.split()

        the_tag = None
        for tag in list(pro_tag_set):
            if tag in lst:
                the_tag = [ele for ele in lst if ele[0] == '_'][0]
                break

        if the_tag != None: # has pronoun in this command
            clean_com_lst[i] = ' '.join([ele for ele in lst if ele[0] != '_'])
            # ------------ store command idx ------------
            pro_dict['idx'].append(i)

            words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
            pro_dict['words'].append(' '.join(words_lst))
            # ------------ store referral, the pronoun in word command ------------
            if 'its' in the_tag:
                pro_loc = words_lst.index('its')
                pro_dict['referral'].append((pro_loc,'its'))
            elif 'it' in the_tag:
                pro_loc = words_lst.index('it')
                pro_dict['referral'].append((pro_loc,'it'))

            # ------------ store referee, the word being referred ------------
            raw_loc = lst.index(the_tag)
            search_lst = lst[:raw_loc][::-1]
            obj_tag_count = 0
            i_count = 0
            rev_referee = []
            # print(search_lst)
            for j,ele in enumerate(search_lst):
                if obj_tag_count == 2:
                    rev_referee.append(ele)
                    if search_lst[j] == '-i-':
                        rev_referee.pop()
                        i_count += 1
                        continue

                    elif i_count ==len(rev_referee): # unfinished storing referee
                        continue

                    else:
                        break

                if '_obj-' in ele:
                    obj_tag_count += 1

            referee_lst = rev_referee[::-1]
            referee_loc = find_sub_list(referee_lst,words_lst)
            str_referee = ' '.join(referee_lst).strip()
            pro_dict['referee'].append((referee_loc,str_referee))

    return pro_dict,clean_com_lst

# get_it_s_lab(com_lst)
pro_dict,clean_com_lst = store_it_s_coref(com_lst)
it_df =pd.DataFrame(pro_dict)
it_df

,idx,referee,referral,words
0,25,"((10, 11), pod)","(14, it)","Please enter to the living room , locate the c..."
1,50,"((3, 4), drinks)","(9, it)",Pick up a drinks from the sink and put it into...
2,60,"((6, 7), paprika)","(9, it)","Could you please pick up the paprika , set it ..."
3,67,"((6, 7), orange)","(13, it)","Look for Alex , grasp the orange from the book..."
4,71,"((6, 7), apple)","(13, it)","Look for Charlie , retrieve the apple from the..."
...,...,...,...,...
7614,99939,"((3, 4), orange)","(6, it)","Pick up the orange , set it on the storage tab..."
7615,99944,"((3, 4), food)","(10, it)",Pick up a food from the dining table and put i...
7616,99955,"((3, 4), snacks)","(9, it)",Pick up a snacks from the bookcase and put it ...
7617,99974,"((2, 3), fruits)","(8, it)","Grasp the fruits from the sink , give it to Al..."


In [6]:
def merge_df(com_lst,it_df,per_df):
    it_idx = it_df['idx'].tolist()
    per_idx = per_df['idx'].tolist()
    dic = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        if i not in it_idx and i not in per_idx:
            dic['idx'].append(i)
            dic['referee'].append('')
            dic['referral'].append('')
            words_lst = [ele for ele in com.split() if ele[0] != '_' and ele[0] != '-']
            dic['words'].append(' '.join(words_lst))

    rest_df = pd.DataFrame(dic)
    df = it_df.copy().append(per_df)
    df = df.append(rest_df)
    df = df.sort_values('idx')
    return df

pro_df = merge_df(clean_com_lst,it_df,per_df)
pro_df

,idx,referee,referral,words
0,0,,,Could you please tell me how the person standi...
1,1,,,Please offer something to drink to all the chi...
2,2,,,Handshake the youngest man in the dining room ...
3,3,,,"Locate a dish in the bedroom , locate Patricia..."
4,4,,,Offer something to drink to all the elders wea...
...,...,...,...,...
85687,99988,,,Tell me how many sitting elders there are in t...
85688,99989,,,"Look for Skyler at the bed , spot Robin at the..."
85689,99990,,,Describe the objects in the dining table to me...
85690,99991,,,Follow the man waving in the corridor


In [7]:
def gen_pro_labels(pro_df):
    referee_lst = pro_df['referee'].tolist()
    referral_lst = pro_df['referral'].tolist()
    words_lst = [ele.split() for ele in pro_df['words'].tolist()]
    pro_labels = [['O']*len(lst) for lst in words_lst]
    for sample_idx in range(len(words_lst)):
        if referral_lst[sample_idx] != '':
            referral_idx = referral_lst[sample_idx][0]
            pro_labels[sample_idx][referral_idx] = 'referral'

            referee_start = referee_lst[sample_idx][0][0]
            referee_end = referee_lst[sample_idx][0][1]
            pro_labels[sample_idx][referee_start] = 'B-referee'
            length = referee_end - referee_start
            if length > 1:
                pro_labels[sample_idx][referee_start + 1 : referee_end] = ['I-referee']*(length - 1)

    return pro_labels
pro_labels = gen_pro_labels(pro_df)
print(pro_labels[:10])

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [8]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):
        # if 'end -i- table' in com:
        #     print(com)
        #     print(i)
        com = com.split()
        if '-i-' not in com:
            res.append(com)
            continue

        #deal with -i-
        elif '-i-' in com:
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')

            while '-i- -' in string:
                string = string.replace('-i- -','-')

            com = string.split()

            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0:
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])

            if i == 1:
                print(com)
                print(idx_lst)

            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                #com[lst[0]] = end_label.replace(',','')
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')#.replace(',','')

                for j,idx in enumerate(lst):
                    if j == 0:
                        com[idx] = end_label ################################################
                    else:
                        label = com[lst[-1]+2]

                        com[idx] = label

            labelled = com.copy()

        res.append(labelled)
    return res

res = improve_labels(clean_com_lst)
# ' '.join(res[3])
res[99849] #989

['Please', 'offer', '-B_take-', 'something', '-B_take_obj-', 'to', '-I_take_obj-', 'drink', '-I_take_obj-', 'to', 'all', '-B_take_dest-', 'the', '-I_take_dest-', 'children', '-i-', 'pointing', '-i-', 'to', '-i-', 'the', '-i-', 'left', '-I_take_dest-', 'in', 'the', 'dining', '-i-', 'room', '-B_take_dest-']
[[15, 17, 19, 21], [27]]


['Retrieve',
 '-B_take-',
 'the',
 'paprika',
 '-B_take_obj-',
 'from',
 'the',
 'end',
 '-B_take_sour-',
 'table',
 '-I_take_sour-',
 ',',
 'hand',
 '-B_take-',
 'it',
 '-B_take_obj-',
 'to',
 'Francis',
 '-B_take_dest-',
 ',',
 'and',
 'answer',
 '-B_answer-',
 'a',
 'question',
 '-B_answer_what-']

In [9]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        clean_out_lst = []
        for ele in com:
#             print('ele; ',ele)
            if ele[0] != '-':
                in_lst.append(ele)
                clean_out_lst.append('O')
            else:
                clean_out_lst[-1] = ele.replace('-','',2)
                if ',' in ele:
                    clean_out_lst[-1] = clean_out_lst[-1].replace(',','')

        seq_in.append(in_lst)
        seq_out.append(clean_out_lst)

    return seq_in, seq_out

seq_in, seq_out = generate_seq_out(res)
print(seq_in[99849])
seq_out[99849]

['Retrieve', 'the', 'paprika', 'from', 'the', 'end', 'table', ',', 'hand', 'it', 'to', 'Francis', ',', 'and', 'answer', 'a', 'question']


['B_take',
 'O',
 'B_take_obj',
 'O',
 'O',
 'B_take_sour',
 'I_take_sour',
 'O',
 'B_take',
 'B_take_obj',
 'O',
 'B_take_dest',
 'O',
 'O',
 'B_answer',
 'O',
 'B_answer_what']

# Check tell commands

In [10]:
for i,(com,lab) in enumerate(zip(seq_in,seq_out)):
    if 'B_tell' in lab:
        print(i)
        print(com)
        print(lab)
        print('-----------------------------')
    if i == 1000:
        break


0
['Could', 'you', 'please', 'tell', 'me', 'how', 'the', 'person', 'standing', 'at', 'the', 'entrance', 'looks', 'like']
['O', 'O', 'O', 'B_tell', 'B_tell_dest', 'O', 'O', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'O', 'O']
-----------------------------
3
['Locate', 'a', 'dish', 'in', 'the', 'bedroom', ',', 'locate', 'Patricia', 'in', 'the', 'kitchen', ',', 'and', 'say', 'something']
['B_find', 'O', 'B_find_obj', 'O', 'O', 'B_find_dest', 'O', 'B_find', 'B_find_per', 'O', 'O', 'B_find_dest', 'O', 'O', 'B_tell', 'B_tell_what']
-----------------------------
5
['Could', 'you', 'tell', 'me', 'how', 'many', 'sitting', 'men', 'there', 'are', 'in', 'the', 'kitchen']
['O', 'O', 'B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what']
-----------------------------
8
['Tell', 'me', 'how', 'many', 'cleaning', 'stuff', 'items', 'are', 'in', 'the', 'storage', 'table

In [11]:
# need to process the pro df as well
def rem_say(seq_in,seq_out,pro_labels):
    say_idx = [i for i,labels in enumerate(seq_out) if 'B_say' in labels]
    seq_in = [seq for i,seq in enumerate(seq_in) if i not in  say_idx]
    seq_out = [seq for i,seq in enumerate(seq_out) if i not in  say_idx]
    seq_pro = [seq for i,seq in enumerate(pro_labels) if i not in  say_idx]
    return seq_in, seq_out, seq_pro

def rem_dups(com_lst, lab_lst, pro_labels,rem = True):
    com_lst = [' '.join(lst) for lst in com_lst]
    lab_lst = [' '.join(lst) for lst in lab_lst]
    if rem:
        com_lst, lab_lst, seq_pro = rem_say(com_lst,lab_lst,pro_labels)
    else:
        seq_pro = pro_labels

    df = pd.DataFrame (com_lst, columns = ['command'])
    df['label'] = lab_lst
    df['pronouns'] = seq_pro
    df2 = df.drop_duplicates(subset=["command"], keep='first')
    return df2

df = rem_dups(seq_in,seq_out,pro_labels, rem = False)
df

,command,label,pronouns
0,Could you please tell me how the person standi...,O O O B_tell B_tell_dest O O B_tell_what I_tel...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,Please offer something to drink to all the chi...,O B_take B_take_obj I_take_obj I_take_obj O B_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Handshake the youngest man in the dining room ...,B_greet O B_greet_per I_greet_per O O B_greet_...,"[O, O, O, O, O, O, O, O, O]"
3,"Locate a dish in the bedroom , locate Patricia...",B_find O B_find_obj O O B_find_dest O B_find B...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,Offer something to drink to all the elders wea...,B_take B_take_obj I_take_obj I_take_obj O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
99985,"Navigate to the corridor , spot Elizabeth , an...",B_go O O B_go_dest O B_find B_find_per O O B_f...,"[O, O, O, O, O, O, O, O, O, O, O]"
99986,"Robot please find Skyler at the dining table ,...",O O B_find B_find_per O O B_find_dest I_find_d...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
99989,"Look for Skyler at the bed , spot Robin at the...",B_find I_find B_find_per O O B_find_dest O B_f...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
99991,Follow the man waving in the corridor,B_follow O B_follow_per I_follow_per O O B_fol...,"[O, O, O, O, O, O, O]"


In [12]:
def write_unique_intent(df):
    label = df['label'].tolist()
    label_lst = [l.split() for l in label]
    # unique_labels
    intents = [ele[2:]  for lst in label_lst for ele in lst  if ele.count('_')==1] #ele[2:] to remove the B_, I_
    #intents.extend(['PAD', 'O'])
    unique_intent_verb = [*set(intents)]
    unique_intent_B = ['B-' + intent for intent in unique_intent_verb]
    unique_intent_I = ['I-' + intent for intent in unique_intent_verb]

    # Interleave
    unique_intent = unique_intent_B + unique_intent_I
    unique_intent[::2] = unique_intent_B
    unique_intent[1::2] = unique_intent_I

    unique_intent = ['PAD','O'] + unique_intent

    with open('intent_label.txt','w') as f:
        for intent in unique_intent:
            f.write(intent)
            f.write('\n')
    return unique_intent

unique_intent = write_unique_intent(df)
unique_intent

['PAD',
 'O',
 'B-greet',
 'I-greet',
 'B-know',
 'I-know',
 'B-follow',
 'I-follow',
 'B-take',
 'I-take',
 'B-tell',
 'I-tell',
 'B-guide',
 'I-guide',
 'B-go',
 'I-go',
 'B-answer',
 'I-answer',
 'B-find',
 'I-find']

# Train Dev Test split

In [13]:
from sklearn.model_selection import train_test_split

def split(df):
    df_train, df_test_dev = train_test_split(df, test_size=0.25, random_state=42)
    df_dev, df_test = train_test_split(df_test_dev, test_size=0.8, random_state=42)

    env_dict = {'train' : df_train,
               'dev' : df_dev,
               'test' : df_test}
    for df in env_dict.values():
        df.columns = ['seq_in', 'seq_out', 'seq_pro']
    return env_dict

env_dict = split(df)
for env,df in env_dict.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_pro
18404,Could you offer something to eat to all the el...,O O B_take B_take_obj I_take_obj I_take_obj O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
17103,Robot please tell me how many elders dressed i...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
27512,Tell me how many men dressed in yellow there a...,B_tell B_tell_dest B_tell_what I_tell_what I_t...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
8788,"Please pinpoint Michael , find Francis in the ...",O B_find B_find_per O B_find B_find_per O O B_...,"[O, O, O, O, O, B-referee, O, O, O, O, O, O, r..."
89493,"Spot Alex in the kitchen , follow him , say so...",B_find B_find_per O O B_find_dest O B_follow B...,"[O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
88070,Robot please bring me the black tableware from...,O O B_take B_take_dest O B_take_obj I_take_obj...,"[O, O, O, O, O, O, O, O, O, O]"
57127,Robot please tell me how many gray orange are ...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
887,Tell me how many orange apple are in the cupboard,B_tell B_tell_dest B_tell_what I_tell_what I_t...,"[O, O, O, O, O, O, O, O, O, O]"
20028,Take the containers at the cupboard's top to t...,B_take O B_take_obj O O B_take_sour I_take_sou...,"[O, O, O, O, O, O, O, O, O, O]"


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro
53026,"Pick up the orange from the bookcase , put app...",B_take I_take O B_take_obj O O B_take_sour O B...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
83739,"Spot a dish in the kitchen , spot Francis , an...",B_find O B_find_obj O O B_find_dest O B_find B...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
89035,"Enter to the dining room , find Linda , and es...",B_go O O B_go_dest I_go_dest O B_find B_find_p...,"[O, O, O, O, O, O, O, O, O, O, O, O]"
1107,"Could you please go to the kitchen , find the ...",O O O B_go O O B_go_dest O B_find O B_find_obj...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
38217,"Look for Alex at the dining table , spot Robin...",B_find I_find B_find_per O O B_find_dest I_fin...,"[O, O, O, O, O, O, O, O, O, B-referee, O, O, O..."
...,...,...,...
87723,"Spot a noodles in the living room , pinpoint S...",B_find O B_find_obj O O B_find_dest I_find_des...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
93196,Say hello to the woman dressed in an orange sw...,B_greet I_greet O O B_greet_per I_greet_per I_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
88592,"Please retrieve the apple from the cupboard , ...",O B_take O B_take_obj O O B_take_sour O B_find...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43021,Offer something to eat to all the women raisin...,B_take B_take_obj I_take_obj I_take_obj O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro
61161,Please tell me how many children in the kitche...,O B_tell B_tell_dest B_tell_what I_tell_what I...,"[O, O, O, O, O, O, O, O, O, O, O, O]"
82166,"Could you retrieve the apple from the sink , p...",O O B_take O B_take_obj O O B_take_sour O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2802,Offer something to eat to all the children wea...,B_take B_take_obj I_take_obj I_take_obj O B_ta...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
46203,Could you please handshake the child in the li...,O O O B_greet O B_greet_per O O B_greet_dest I...,"[O, O, O, O, O, O, O, O, O, O]"
97607,"Robot please go to the bookcase , find Alex , ...",O O B_go O O B_go_dest O B_find B_find_per O O...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
77840,Robot please salute the slimmest man in the li...,O O B_greet O B_greet_per I_greet_per O O B_gr...,"[O, O, O, O, O, O, O, O, O, O]"
12540,Robot please pour some grape juice in a bowl,O O B_take O B_take_obj I_take_obj O O B_take_...,"[O, O, O, O, O, O, O, O, O]"
29193,"Take the orange , put it on the side table , a...",B_take O B_take_obj O B_take B_take_obj O O B_...,"[O, O, B-referee, O, O, referral, O, O, O, O, ..."
50047,"Meet Robin at the dining table , then greet he...",B_find B_find_per O O B_find_dest I_find_dest ...,"[O, B-referee, O, O, O, O, O, O, O, referral, O]"


------------------------------------------------------------------


In [14]:
def find_sub_list(subl, l):
    # s_str = ' '.join(subl)
    # f_str = ' '.join(l)
    # num = f_str.count(s_str)
    # print(num)
    # ind_subl = []
    # for i in range(num):
    #     ind_subl.extend([i for i in range(len(l)) if l[i] in subl])
    ind_subl = [i for i in range(len(l)) if l[i] in subl]

    if len(ind_subl) == 1:
        return [ind_subl[0]]
    else:
        # if len(ind_subl) > len(l):
        #     for i in range(len(ind_subl)-len(l)):

        return ind_subl

l=['O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what']
find_sub_list(['I-what', 'O', 'I-what'],l)


[0, 3, 4, 5, 6, 7, 8, 9, 10]

In [15]:
def seq_intent_out(com_lst,lab_lst):
    out_int_lst = []

    #polite = ['Please','Could you', 'Robot please', 'Could you please', 'please']
    polite_unique = ['Please','Could you', 'Robot', 'please']

    #sep = ['and', ',' , ', and']
    sep_unique = ['and', ',' ]
    for idx,(com,lab) in enumerate(zip(com_lst,lab_lst)):

        sentence = ' '.join(com)

        polite_dict = {key:find_sub_list(key.split(' '),com) for key in polite_unique if key in sentence}
        polite_idx = []
        for ele in polite_dict.values():
            polite_idx.extend(ele)

        sep_dict = {key:find_sub_list(key.split(' '),com) for key in sep_unique if key in sentence}
        sep_idx = []
        for ele in sep_dict.values():
            sep_idx.extend(ele)

        non_intent_idx = polite_idx + sep_idx
        lst = ['O' if i in non_intent_idx else None for i in range(len(lab))]

        if 'get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'Get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['Get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'then' in com:
            j = com.index('then')
            lst[j] = 'O'


        intent_lst = []
        B = True
        for i,ele in enumerate(lst):
            if ele == None and B ==True:
                B = False
                if lab[i] != 'O':
                    intent = lab[i].split('_')[1]
                else:
                    intent = intent_lst[-1][0]
                intent_lst.append([intent,[i]])
            elif ele == None and B == False:
                intent_lst[-1][1].append(i)

            elif ele == 'O':
                B = True

        intent_seq = lst#[ele if ele.count('_') != 1 else ele.replace('_','-') for ele in lab]
        for instance_lst in intent_lst:
            # for subtask_lst in instance_lst:
            intent, idx_lst =  instance_lst[0], instance_lst[1]
            begin = idx_lst.pop(0)
            intent_seq[begin] = 'B-' + intent
            for i in idx_lst:
                intent_seq[i] = 'I-' + intent

        temp = ' '.join(intent_seq)
        if 'B-tell I-tell O B-tell' in temp:
            intent_seq = temp.replace('B-tell I-tell O B-tell','B-tell I-tell I-tell I-tell').split()
            # print(com)
            # print(intent_seq)



        out_int_lst.append(intent_seq)

        # if idx == 8:
        if 'I_tell_what' in intent_seq:
            print('  '.join(com))
            print('lab: ', lab)
            print('polite_dict: ',polite_dict)
            print('sep_dict: ',sep_dict)
            print('lst: ', lst)
            print('intent_lst: ',intent_lst)
            print(intent_seq)
            print('-----------------------------------------------------------')
    return out_int_lst


#
lab_lst = [ele.split() for ele in env_dict['train']['seq_out'].tolist()]
com_lst = [ele.split() for ele in env_dict['train']['seq_in'].tolist()]
seq_intent_out(com_lst,lab_lst)

[['O',
  'O',
  'B-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take'],
 ['O',
  'O',
  'B-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell'],
 ['B-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell'],
 ['O',
  'B-find',
  'I-find',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'O',
  'O',
  'B-follow',
  'I-follow'],
 ['B-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'O',
  'B-follow',
  'I-follow',
  'O',
  'B-tell',
  'I-tell'],
 ['O',
  'O',
  'B-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'I-find',
  'O',
  'B-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'O',
  'B-take',
  'I-take',
  'I-take'],
 ['B-fi

In [16]:
a = 'I-what O I-what'
b = a.replace('O','I-what')
a

'I-what O I-what'

In [17]:
def write(f_name,data_lst, space = True):
    with open(f_name, 'w') as f:
        for i,lst in enumerate(data_lst):
            if space == True:
                # print(i)
                # print(lst)
                com = ' '.join(lst)
            else:
                com = ''.join(lst)
            f.write(com)
            f.write('\n')
    return

def gen_files(env_name, data_df):
    # get seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]

    #get seq in
    com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]
    #get pro labels lst
    pro_labels = data_df['seq_pro'].tolist()

    int_lst = seq_intent_out(com_lst,lab_lst)

    # get label
    label = []
    for lab_seq in int_lst:
        lst = []
        for lab in lab_seq:
            if lab == 'O':
                continue
            elif len(lst) == 0:
                lst.append(lab)
            elif lab not in lst:
                lst.append(lab)
        hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
        label.append(hash_taged)


    # get seq out
    # 1. remove the intent label out of the seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
    # 2. remove the intentions names in the slot names
    slot_seq = []
    for ite,one_seq in enumerate(lab_lst):
        # if 'end table' in ' '.join(com_lst[ite]):
        #     print(ite)
        #     print(' '.join(com_lst[ite]))
        lst = []
        for slot in one_seq:
            if slot == 'O':
                lst.append('O')
            else:
                # with IBO
                slot_start = slot.find('_',2)+1
                entity_type = slot[slot_start:]
                lst.append(slot[0]+'-'+entity_type)


        temp = ' '.join(lst)
        if 'I-what O I-what' in temp:
            lst = temp.replace('I-what O I-what','I-what I-what I-what').split()

        if 'I-what O I-what' in temp:
            lst = temp.replace('I-what O I-what','I-what I-what I-what').split()


        if 'B-per B-per' in ' '.join(lst):
            print(ite)
            print(com_lst[ite])
            print(lst)

        if ite == 12054:
            print(com_lst[ite])
            print(lst)

        slot_seq.append(lst)




    df = data_df.copy()
    df['label'] = label
    df['seq_intent'] = int_lst
    df['seq_out'] = slot_seq
    df['seq_pro'] = pro_labels


    # write all files
    write(f'./{env_name}/seq.in',com_lst)
    write(f'./{env_name}/seq.out',slot_seq)
    write(f'./{env_name}/seq_intent.out',int_lst)
    write(f'./{env_name}/seq_pro.out',pro_labels)
    write(f'./{env_name}/label',label, False)

    return df

df_train_all = gen_files('train',env_dict['train'])
df_dev_all = gen_files('dev',env_dict['dev'])
df_test_all = gen_files('test',env_dict['test'])

env_dict_all = {'train' : df_train_all,
               'dev' : df_dev_all,
               'test' : df_test_all}

for env,df in env_dict_all.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

['Robot', 'please', 'retrieve', 'the', 'orange', 'from', 'the', 'storage', 'table', ',', 'locate', 'a', 'pringles', ',', 'and', 'tell', 'whether', 'you', 'dream', 'or', 'not', 'on', 'electric', 'sheep']
['O', 'O', 'O', 'O', 'B-obj', 'O', 'O', 'B-sour', 'I-sour', 'O', 'O', 'O', 'B-obj', 'O', 'O', 'O', 'B-what', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what']
['Get', 'to', 'know', 'Alex', 'at', 'the', 'end', 'table', ',', 'then', 'greet', 'her']
['O', 'O', 'O', 'B-per', 'O', 'O', 'B-dest', 'I-dest', 'O', 'O', 'O', 'B-per']
train 



,seq_in,seq_out,seq_pro,label,seq_intent
18404,Could you offer something to eat to all the el...,"[O, O, O, B-obj, I-obj, I-obj, O, B-dest, I-de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
17103,Robot please tell me how many elders dressed i...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
27512,Tell me how many men dressed in yellow there a...,"[O, B-dest, B-what, I-what, I-what, I-what, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[B-tell, I-tell, I-tell, I-tell, I-tell, I-tel..."
8788,"Please pinpoint Michael , find Francis in the ...","[O, O, B-per, O, O, B-per, O, O, B-dest, O, O,...","[O, O, O, O, O, B-referee, O, O, O, O, O, O, r...","[B-find#, I-find#, B-follow#, I-follow]","[O, B-find, I-find, O, B-find, I-find, I-find,..."
89493,"Spot Alex in the kitchen , follow him , say so...","[O, B-per, O, O, B-dest, O, O, B-per, O, O, B-...","[O, O, O, O, O, O, O, O, O, O, O]","[B-find#, I-find#, B-follow#, I-follow#, B-tel...","[B-find, I-find, I-find, I-find, I-find, O, B-..."
...,...,...,...,...,...
88070,Robot please bring me the black tableware from...,"[O, O, O, B-dest, O, B-obj, I-obj, O, O, B-sour]","[O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
57127,Robot please tell me how many gray orange are ...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
887,Tell me how many orange apple are in the cupboard,"[O, B-dest, B-what, I-what, I-what, I-what, I-...","[O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[B-tell, I-tell, I-tell, I-tell, I-tell, I-tel..."
20028,Take the containers at the cupboard's top to t...,"[O, O, B-obj, O, O, B-sour, I-sour, O, O, B-dest]","[O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro,label,seq_intent
53026,"Pick up the orange from the bookcase , put app...","[O, O, O, B-obj, O, O, B-sour, O, O, B-obj, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
83739,"Spot a dish in the kitchen , spot Francis , an...","[O, O, B-obj, O, O, B-dest, O, O, B-per, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-find#, I-find#, B-tell#, I-tell]","[B-find, I-find, I-find, I-find, I-find, I-fin..."
89035,"Enter to the dining room , find Linda , and es...","[O, O, O, B-dest, I-dest, O, O, B-per, O, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O]","[B-go#, I-go#, B-find#, I-find#, B-guide#, I-g...","[B-go, I-go, I-go, I-go, I-go, O, B-find, I-fi..."
1107,"Could you please go to the kitchen , find the ...","[O, O, O, O, O, O, B-dest, O, O, O, B-obj, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-go#, I-go#, B-find#, I-find#, B-tell]","[O, O, O, B-go, I-go, I-go, I-go, O, B-find, I..."
38217,"Look for Alex at the dining table , spot Robin...","[O, O, B-per, O, O, B-dest, I-dest, O, O, B-pe...","[O, O, O, O, O, O, O, O, O, B-referee, O, O, O...","[B-find#, I-find#, B-guide#, I-guide]","[B-find, I-find, I-find, I-find, I-find, I-fin..."
...,...,...,...,...,...
87723,"Spot a noodles in the living room , pinpoint S...","[O, O, B-obj, O, O, B-dest, I-dest, O, O, B-pe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find#, B-guide#, I-guide]","[B-find, I-find, I-find, I-find, I-find, I-fin..."
93196,Say hello to the woman dressed in an orange sw...,"[O, O, O, O, B-per, I-per, I-per, I-per, I-per...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-greet#, I-greet]","[B-greet, I-greet, I-greet, I-greet, I-greet, ..."
88592,"Please retrieve the apple from the cupboard , ...","[O, O, O, B-obj, O, O, B-sour, O, O, B-per, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[O, B-take, I-take, I-take, I-take, I-take, I-..."
43021,Offer something to eat to all the women raisin...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro,label,seq_intent
61161,Please tell me how many children in the kitche...,"[O, O, B-dest, B-what, I-what, I-what, I-what,...","[O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, B-tell, I-tell, I-tell, I-tell, I-tell, I-..."
82166,"Could you retrieve the apple from the sink , p...","[O, O, O, O, B-obj, O, O, B-sour, O, O, B-obj,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
2802,Offer something to eat to all the children wea...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
46203,Could you please handshake the child in the li...,"[O, O, O, O, O, B-per, O, O, B-dest, I-dest]","[O, O, O, O, O, O, O, O, O, O]","[B-greet#, I-greet]","[O, O, O, B-greet, I-greet, I-greet, I-greet, ..."
97607,"Robot please go to the bookcase , find Alex , ...","[O, O, O, O, O, B-dest, O, O, B-per, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-go#, I-go#, B-find#, I-find#, B-guide#, I-g...","[O, O, B-go, I-go, I-go, I-go, O, B-find, I-fi..."
...,...,...,...,...,...
77840,Robot please salute the slimmest man in the li...,"[O, O, O, O, B-per, I-per, O, O, B-dest, I-dest]","[O, O, O, O, O, O, O, O, O, O]","[B-greet#, I-greet]","[O, O, B-greet, I-greet, I-greet, I-greet, I-g..."
12540,Robot please pour some grape juice in a bowl,"[O, O, O, O, B-obj, I-obj, O, O, B-dest]","[O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
29193,"Take the orange , put it on the side table , a...","[O, O, B-obj, O, O, B-obj, O, O, B-dest, I-des...","[O, O, B-referee, O, O, referral, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[B-take, I-take, I-take, O, B-take, I-take, I-..."
50047,"Meet Robin at the dining table , then greet he...","[O, B-per, O, O, B-dest, I-dest, O, O, O, B-pe...","[O, B-referee, O, O, O, O, O, O, O, referral, O]","[B-find#, I-find#, B-greet#, I-greet]","[B-find, I-find, I-find, I-find, I-find, I-fin..."


------------------------------------------------------------------


In [32]:
for i,slot in enumerate(env_dict_all['train']['seq_out'].tolist()):
    lab = ['O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab1 = ['O', 'O', 'O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab2 = ['O', 'O', 'O', 'B-obj', 'O', 'O', 'B-sour', 'O', 'O', 'O', 'B-obj', 'O', 'B-dest', 'O', 'O', 'B-dest', 'O', 'O', 'O', 'B-obj', 'I-obj']
    if ' '.join(slot) == ' '.join(lab) or ' '.join(slot) == ' '.join(lab2) or ' '.join(slot) == ' '.join(lab1):
        print(env_dict_all['train']['seq_in'].tolist()[i])


for i,slot in enumerate(env_dict_all['train']['seq_out'].tolist()):
    lab = ['O', 'O', 'B-what', 'O', 'O', 'I-what', 'O', 'O', 'B-what', 'O', 'O', 'O', 'B-per', 'I-per', 'I-per', 'I-per']
    lab1 = ['O', 'B-what', 'O', 'O', 'I-what', 'O', 'O', 'O', 'B-what', 'O', 'O', 'O', 'B-per', 'I-per', 'I-per', 'I-per', 'I-per', 'I-per', 'I-per', 'I-per']
    if ' '.join(slot) == ' '.join(lab1):
        print(env_dict_all['train']['seq_in'].tolist()[i])



for i,intent in enumerate(env_dict_all['dev']['seq_intent'].tolist()):
    lab = ['O', 'O', 'B-know', 'I-know', 'O', 'B-guide', 'I-guide', 'I-guide', 'I-guide', 'I-guide', 'I-guide', 'I-guide', 'O', 'O', 'B-find', 'I-find', 'O', 'B-find', 'I-find', 'I-find', 'I-find', 'I-find', 'I-find']
    if ' '.join(intent) == ' '.join(lab):
        print(env_dict_all['train']['seq_in'].tolist()[i])

In [ ]:
def write_unique_slot(env_dict_all):
    label = []
    for df in env_dict_all.values():
        label.extend(df['seq_out'].tolist())

    label_lst = label.copy()

    slots = [ele  for lst in label_lst for ele in lst if ele != 'O'] #ele[2:] to remove the B_, I_
    unique_slot = ['PAD','O'] + [*set(slots)]

    with open('slot_label.txt','w') as f:
        for slot in unique_slot:
            f.write(slot)
            f.write('\n')
    return unique_slot

unique_slot = write_unique_slot(env_dict_all)
unique_slot

In [ ]:
def show_unique_labels(env):
    # uni_slot = [line.strip() for line in open('slot_label.txt', 'r', encoding='utf-8')]
    # uni_intent = [line.strip() for line in open('intent_label.txt', 'r', encoding='utf-8')]

    slot_path =  f'./{env}/seq.out'
    intent_path = f'./{env}/seq_intent.out'

    slot_seq = [line.strip().split() for line in open(slot_path, 'r', encoding='utf-8')]
    intent_seq = [line.strip().split() for line in open(intent_path, 'r', encoding='utf-8')]

    slot_lst = [label for seq in slot_seq for label in seq]
    intent_lst = [label for seq in intent_seq for label in seq]

    env_uni_slot = [*set(slot_lst)]
    env_uni_intent = [*set(intent_lst)]
    print('env: ',env)
    print('Unique intent: ',env_uni_intent,len(env_uni_intent))
    print('Unique slots:  ',env_uni_slot,len(env_uni_slot),'\n')
    return


for env in env_dict_all.keys():
    show_unique_labels(env)